In [1]:
import sys
sys.path.insert(0, '../../')

In [2]:
import bd_calc
import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import topo
import typing
import utils
import yaml

matplotlib.rc('font', family='Times New Roman', size=18)

In [3]:
import all_cloud_cut
import best_cut
import flow_cut
import greedy_cut

def unit_size_cb(r: int):
    return 10000 * math.pow(10, random.randint(0, 1))

In [4]:
topo_file = "../../samples/1e3h.yaml"
source_selector_dict = {'rasp1': 8, 'rasp2': 8, 'rasp3': 8}

sc = topo.Scenario.from_dict(
    yaml.load(open(topo_file, "r").read(), Loader=yaml.Loader)
)

In [6]:
cnt = 0
while cnt < 1000:
    source_selector = graph.SourceSelector(source_selector_dict)
    gen_args = [
        {
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(1, 2),
            "max_predecessors": random.randint(1, 2),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
        }
        for _ in range(4)
    ]
    gs = [graph.GraphGenerator("g"+str(idx), **args).gen_dag_graph() for idx, args in enumerate(gen_args)]
    fr = flow_cut.flow_cut(sc, gs)
    g2r = greedy_cut.greedy_cut2(sc, gs)
    br = best_cut.best_cut(sc, gs)
    fbd = sum([bd_calc.bd_calc(g, r[0], r[1]) for g, r in zip(gs, fr)])
    gbd = sum([bd_calc.bd_calc(g, r[0], r[1]) for g, r in zip(gs, g2r)])
    bbd = sum([bd_calc.bd_calc(g, r[0], r[1]) for g, r in zip(gs, br)])
    # gbd = bd_calc.bd_calc(g, g2r[0], g2r[1])
    if fbd < bbd:
        print("better than best")
        print(fbd, bbd)
        print(cnt)
        break
    if gbd < fbd:
        print(fbd, gbd)
        print(cnt)
        break
    cnt += 1
    # greedy2_result = greedy_cut.greedy_cut2(sc, [g])[0]
    # greedy2_bd = bd_calc.bd_calc(g, greedy2_result[0], greedy2_result[1])
    # all_cloud_result = all_cloud_cut.all_cloud_cut(sc, [g])[0]
    # all_cloud_bd = bd_calc.bd_calc(g, all_cloud_result[0], all_cloud_result[1])
    # if greedy2_bd > all_cloud_bd:
    #     for u, v, d in g.get_edges():
    #         print('{} -- {} --> {}'.format(u, int(d['unit_size']*d['per_second']/1e3), v))
    #     print(greedy2_bd / 1e3)
    #     print(all_cloud_bd / 1e3)
    #     print(greedy2_result)
    #     print(all_cloud_result)
    #     break

9160000.0 8460000.0
107


In [6]:
with open("debug/g4.yaml") as f:
    gs = graph.ExecutionGraph.load_all(f)

In [7]:
for u, v, d in gs[2].get_edges():
    print("{} -- {} --> {}".format(u, int(d['unit_size']*d['per_second']/1e3), v))

g2-v0 -- 200 --> g2-v1
g2-v0 -- 1100 --> g2-v5
g2-v1 -- 150 --> g2-v2
g2-v2 -- 1600 --> g2-v3
g2-v3 -- 140 --> g2-v4
g2-v3 -- 1800 --> g2-v6
g2-v4 -- 1300 --> g2-v5
g2-v5 -- 120 --> g2-v6


In [8]:
# sc = topo.Scenario.from_dict(
#     yaml.load(open("../../samples/1e12h.yaml", "r").read(), Loader=yaml.Loader)
# )
# fr = flow_cut.flow_cut(sc, [g])

In [9]:
fr = flow_cut.flow_cut(sc, [gs[2]])
for r in fr:
    print(r)

fsource->g2-v0,g2-v0->g2-v5,g2-v5->g2-v6,g2-v6->fsink
fsource->g2-v0,g2-v0->g2-v1,g2-v1->g2-v2,g2-v2->g2-v3,g2-v3->g2-v6,g2-v6->fsink
fsource->g2-v0,g2-v0->g2-v5,g2-v5->g2-v4,g2-v4->g2-v3,g2-v3->g2-v6,g2-v6->fsink

{'g2-v4', 'g2-v0', 'g2-v5', 'g2-v1'} {'g2-v2', 'g2-v3', 'g2-v6'}
fsource->g2-v4,g2-v4->g2-v5,g2-v5->fsink
fsource->g2-v0,g2-v0->g2-v1,g2-v1->fsink
fsource->g2-v0,g2-v0->g2-v5,g2-v5->fsink

{'g2-v0', 'g2-v4'} {'g2-v2', 'g2-v3', 'g2-v1', 'g2-v5', 'g2-v6'}
fsource->g2-v0,g2-v0->fsink
fsource->g2-v4,g2-v4->fsink

set() {'g2-v1', 'g2-v5', 'g2-v4', 'g2-v3', 'g2-v2', 'g2-v0', 'g2-v6'}
(set(), {'g2-v1', 'g2-v5', 'g2-v4', 'g2-v3', 'g2-v2', 'g2-v0', 'g2-v6'})


In [10]:
# with open("debug/g4.yaml", "w") as f:
#     graph.ExecutionGraph.save_all(gs, f)

In [9]:
for r in fr:
    print(r)
print('...')
for r in br:
    print(r)

({'g0-v2', 'g0-v0', 'g0-v1'}, {'g0-v3'})
({'g1-v4', 'g1-v3', 'g1-v0', 'g1-v2', 'g1-v1'}, {'g1-v6', 'g1-v5'})
({'g2-v1', 'g2-v0'}, {'g2-v2'})
({'g3-v4', 'g3-v1', 'g3-v0', 'g3-v5', 'g3-v2'}, {'g3-v3'})
...
({'g0-v0', 'g0-v1'}, {'g0-v2', 'g0-v3'})
({'g1-v4', 'g1-v0', 'g1-v2', 'g1-v1'}, {'g1-v6', 'g1-v5', 'g1-v3'})
({'g2-v0', 'g2-v1'}, {'g2-v2'})
({'g3-v1', 'g3-v4', 'g3-v0', 'g3-v2'}, {'g3-v5', 'g3-v3'})


In [10]:
for g in gs:
    for u, v, d in g.get_edges():
        print("{} -- {} --> {}".format(u, int(d['unit_size']*d['per_second']/1e3), v))
    print('---')

g0-v0 -- 180 --> g0-v1
g0-v0 -- 1600 --> g0-v2
g0-v1 -- 110 --> g0-v3
---
g1-v0 -- 160 --> g1-v1
g1-v0 -- 1900 --> g1-v3
g1-v1 -- 190 --> g1-v2
g1-v1 -- 1400 --> g1-v4
g1-v2 -- 130 --> g1-v4
g1-v4 -- 130 --> g1-v5
g1-v5 -- 1100 --> g1-v6
---
g2-v0 -- 1600 --> g2-v1
g2-v0 -- 130 --> g2-v2
g2-v1 -- 1100 --> g2-v2
---
g3-v0 -- 1500 --> g3-v1
g3-v1 -- 1100 --> g3-v2
g3-v1 -- 110 --> g3-v3
g3-v1 -- 190 --> g3-v4
g3-v1 -- 100 --> g3-v5
g3-v2 -- 180 --> g3-v4
g3-v4 -- 120 --> g3-v5
---
